In [6]:
from google.colab import drive
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import json

In [5]:
drive.mount('/content/drive')
# data를 저장할 폴더 입니다. 환경에 맞게 수정 하세요.
data_dir = "/content/drive/My Drive/2022 AI SW 온라인 실무 교육 사례 대회/data/train/TL_"
test_dir = "/content/drive/My Drive/2022 AI SW 온라인 실무 교육 사례 대회/data/test/VL_"

Mounted at /content/drive


In [21]:
def file_extract(data_dir, essay_type):
    dir = data_dir + essay_type + '/' + essay_type + '/'
    file_list = os.listdir(dir)
    
    df = pd.DataFrame(columns=['essay_main_subject', 'essay_prompt', 'paragraph', 'essay_scoreT_org', 'essay_scoreT_cont', 'essay_scoreT_exp'])
    
    for file in tqdm(file_list):
        with open(dir + file) as f:
            js = json.loads(f.read())
            df.loc[len(df)] = [js['info']['essay_main_subject'], 
                               js['info']['essay_prompt'], js['paragraph'][0]['paragraph_txt'], 
                               js['score']['essay_scoreT_detail']['essay_scoreT_org'],
                               js['score']['essay_scoreT_detail']['essay_scoreT_cont'],
                               js['score']['essay_scoreT_detail']['essay_scoreT_exp']] 
    return df

In [22]:
essay_types = os.listdir("/content/drive/My Drive/2022 AI SW 온라인 실무 교육 사례 대회/data/train/")
essay_types = list(map(lambda x: x[3:], essay_types))
essay_types

['주장', '설명글', '찬성반대', '대안제시', '글짓기']

In [23]:
def score_avg(x, len_x):
    avg = []
    for i in range(len_x):
        avg.append(round(sum(np.array(x).T[i])/3))
    return avg

In [ ]:
for e_type in tqdm(essay_types):
    train_essay = file_extract(data_dir, e_type)
    test_essay = file_extract(test_dir, e_type)

    train_essay['essay_prompt'] = train_essay['essay_prompt'].str.replace('\n\n','')
    test_essay['essay_prompt'] = test_essay['essay_prompt'].str.replace('\n\n','')
    train_essay['paragraph'] = train_essay['paragraph'].str.replace('#@문장구분#','')
    test_essay['paragraph'] = test_essay['paragraph'].str.replace('#@문장구분#','')

    y_col = ['essay_scoreT_org', 'essay_scoreT_cont', 'essay_scoreT_exp']
    y_col_fin = []

    for yc in y_col:
        len_x = len(train_essay[yc][0][0])
        train_essay[yc] = train_essay[yc].apply(lambda x: score_avg(x, len_x))
        test_essay[yc] = test_essay[yc].apply(lambda x: score_avg(x, len_x))

        for i in range(len_x):
            col_name = yc+'_'+str(i)
            y_col_fin.append(col_name)
            train_essay[col_name] = train_essay[yc].apply(lambda x: x[i])
            test_essay[col_name] = test_essay[yc].apply(lambda x: x[i])

    train_essay.to_csv(data_dir[:51]+"data/train/train_"+e_type+".csv", index=False)
    test_essay.to_csv(data_dir[:51]+"data/test/test_"+e_type+".csv", index=False)

100%|██████████| 8596/8596 [04:24<00:00, 32.47it/s] 

100%|██████████| 14575/14575 [08:09<00:00, 29.80it/s]

100%|██████████| 6687/6687 [04:08<00:00, 26.86it/s] 

100%|██████████| 5159/5159 [03:18<00:00, 26.01it/s] 

 47%|████▋     | 2161/4574 [02:29<00:19, 125.32it/s]